In [15]:
import pandas as pd
from PIL import Image
import torch
import statsmodels.api as sm
import numpy as np

In [27]:
import math
def Gains_report(actual,preds,n_bins=10):
        actual_and_preds_df=pd.DataFrame()
        actual_and_preds_df['actual']=actual
        actual_and_preds_df['preds']=preds
        one_bin_number=math.ceil(actual_and_preds_df.shape[0]*1.0/n_bins)
        actual_and_preds_df['random_num']=range(actual_and_preds_df.shape[0])
        actual_and_preds_df=actual_and_preds_df.sort_values(by=['preds','random_num'],ascending=False)
        good=[]
        bad=[]
        Mean_Score=[]
        good_temp=0
        bad_temp=0
        binstemp=[]
        iindex= [ int(i * one_bin_number) for i in range(n_bins)]
        iindex.append((actual_and_preds_df.shape[0]))
        for i in range(len(iindex)-1):
            bad.append(actual_and_preds_df.iloc[iindex[i]:iindex[i+1]]['actual'].sum())
            good.append(actual_and_preds_df.iloc[iindex[i]:iindex[i+1]]['actual'].count()-actual_and_preds_df.iloc[iindex[i]:iindex[i+1]]['actual'].sum())
            Mean_Score.append(actual_and_preds_df.iloc[iindex[i]:iindex[i+1]]['preds'].mean())
            binstemp.append(actual_and_preds_df.iloc[iindex[i]]['preds'])
        binstemp.append(actual_and_preds_df.iloc[(actual_and_preds_df.shape[0]-1)]['preds'])
        n = len(binstemp) - 1
        actual=np.array(actual).reshape(-1,1)
        bad_actual = float(sum(actual)[0])
        good_actual = len(actual) - bad_actual
        #np.logical_and是逻辑与只是提供逻辑与的判断，不提供别的信息
        #n:0:-1是生成序列，步长为负1
        Total_No_Bad_percent=np.array(good)/good_actual*100
        Total_Bads_percent=np.array(bad)/bad_actual*100
        Total_No_Bad_percent_str=pd.Series(Total_No_Bad_percent).apply(lambda x: str(np.round(x,2))+'%' if(x!=0) else '0').values
        Total_Bads_percent_str=pd.Series(Total_Bads_percent).apply(lambda x: str(np.round(x,2))+'%' if(x!=0) else '0').values
        Cum_Total_Non_Bad=[sum(Total_No_Bad_percent[0:i+1]) for i in range(len(Total_No_Bad_percent))]
        Cum_Total_Bads=[sum(Total_Bads_percent[0:i+1]) for i in range(len(Total_Bads_percent))]
        Cum_Total_Bads_str=pd.Series(Cum_Total_Bads).apply(lambda x: str(np.round(x,2))+'%' if(x!=0) else '0').values
        Cum_Total_Non_Bad_str=pd.Series(Cum_Total_Non_Bad).apply(lambda x: str(np.round(x,2))+'%' if(x!=0) else '0').values
                                              
        Bads_vs_Non_Bad_KS=np.array(Cum_Total_Bads)-np.array(Cum_Total_Non_Bad)
        Bads_vs_Non_Bad_KS_str=pd.Series(Bads_vs_Non_Bad_KS).apply(lambda x: str(np.round(x,2))+'%' if(x!=0) else '0').values 
        total = [x[0]+x[1] for x in zip(bad, good)]#这里写的很好
        Interval_Bads_Rate=np.array(bad)*1.0/np.array(total)
        Interval_Bads_Rate=pd.Series(Interval_Bads_Rate).apply(lambda x: str(np.round(x*100,2))+'%' if(x!=0) else '0').values                         
        gains_report = pd.DataFrame(pd.np.zeros([len(binstemp)-1, 12]),index=range(1,len(binstemp)),columns=["# Total","Max Score","Min Score","Mean Score","# Non-Bad","% Total Non-Bad","Cum % Total Non-Bad","# Bads","% Total Bads","% Cum % Total Bads","Interval Bads Rate","Bads vs. Non-Bad K-S"])
        (gains_report["# Total"],gains_report["Max Score"],gains_report["Min Score"],gains_report["Mean Score"],gains_report["# Non-Bad"],gains_report["% Total Non-Bad"],gains_report['Cum % Total Non-Bad'],gains_report["# Bads"],gains_report["% Total Bads"],gains_report["% Cum % Total Bads"],gains_report["Interval Bads Rate"],gains_report["Bads vs. Non-Bad K-S"]) = (total,binstemp[0:-1],binstemp[1:],Mean_Score,good,Total_No_Bad_percent_str,Cum_Total_Non_Bad_str,bad,Total_Bads_percent_str,Cum_Total_Bads_str,Interval_Bads_Rate,Bads_vs_Non_Bad_KS_str)
        gains_report.reset_index(inplace=True)
        gains_report.rename(index=str, columns={"index": "Rank"},inplace=True)
        gains_report.loc[n_bins]={"Rank":'Total',"# Total":gains_report['# Total'].sum(),"Max Score":gains_report["Max Score"].max(),"Min Score":gains_report["Min Score"].min(),"Mean Score":np.mean(preds),"# Non-Bad":gains_report["# Non-Bad"].sum(),"% Total Non-Bad":str(np.round(Total_No_Bad_percent.sum(),2))+'%',"Cum % Total Non-Bad":"","# Bads":gains_report["# Bads"].sum(),"% Total Bads":str(np.round(Total_Bads_percent.sum(),2))+'%',"% Cum % Total Bads":"","Interval Bads Rate":str(np.round((len(preds) - good_actual)/len(preds)*100,2))+'%',"Bads vs. Non-Bad K-S":str(np.round(Bads_vs_Non_Bad_KS.max(),2))+'%'}
        return  gains_report

In [42]:
picturescore=pd.read_csv(r'/home/tongqing.li/qianbao/deep_learning/neural_network_in_pytorch/data/picturescore_new.csv')

In [74]:
pictureprobscore = pd.read_csv(r'/home/tongqing.li/qianbao/deep_learning/neural_network_in_pytorch/data/picture_score_probability.csv')

In [75]:
pictureprobscore=pictureprobscore.rename(columns={'Heavy_Makeup':'Heavy_Makeup_prob_score'})
pictureprobscore

,Unnamed: 0,filename,applied_id,Eyeglasses,Heavy_Makeup_prob_score,Male,Wearing_Earrings,Wearing_Necktie,Attractive,Wearing_Hat
0,0,52697741_04.jpg,52697741,0.014214,0.002846,0.678494,0.045963,0.000709,0.007449,0.183183
1,1,52697322_04.jpg,52697322,0.133729,0.018194,0.524594,0.169617,0.001598,0.063551,0.006050
2,2,52697214_04.jpg,52697214,0.021479,0.003544,0.743967,0.037626,0.015387,0.006916,0.006613
3,3,52697150_04.jpg,52697150,0.007787,0.004059,0.761579,0.025865,0.020372,0.005209,0.002456
4,4,52697100_04.jpg,52697100,0.002869,0.386678,0.048841,0.010700,0.000313,0.484459,0.004705
5,5,52697044_04.jpg,52697044,0.020528,0.007459,0.739867,0.079240,0.011732,0.030167,0.022953
6,6,52696984_04.jpg,52696984,0.061803,0.008022,0.903979,0.085382,0.032821,0.006907,0.010603
7,7,52696934_04.jpg,52696934,0.058225,0.003363,0.932688,0.042110,0.014031,0.010334,0.066297
8,8,52696760_04.jpg,52696760,0.016646,0.018532,0.646486,0.028593,0.039018,0.049460,0.040946
9,9,52696225_04.jpg,52696225,0.047805,0.004365,0.773150,0.013750,0.007592,0.018462,0.138002


In [83]:
pictureprobscore_03 = pd.read_csv(r'/home/tongqing.li/qianbao/deep_learning/neural_network_in_pytorch/data/picture_score_probability_03.csv')

In [84]:
pictureprobscore_03=pictureprobscore_03.rename(columns={'Heavy_Makeup':'Heavy_Makeup_prob_score_03'})
pictureprobscore_03

,Unnamed: 0,filename,applied_id,filename1,Eyeglasses,Heavy_Makeup_prob_score_03,Male,Wearing_Earrings,Wearing_Necktie,Attractive,Wearing_Hat,Eyeglasses_flag,Heavy_Makeup_flag,Male_flag,Wearing_Earrings_flag,Wearing_Necktie_flag,Attractive_flag,Wearing_Hat_flag
0,0,52697741_04.jpg,52697741,52697741_03.jpg,0.000674,0.002308,0.302895,0.025466,0.001164,0.000332,0.001025,0,0,1,0,0,0,0
1,1,52697322_04.jpg,52697322,52697322_03.jpg,0.003457,0.005363,0.292629,0.017999,0.008839,0.003745,0.000528,0,0,1,0,0,0,0
2,2,52697214_04.jpg,52697214,52697214_03.jpg,0.000774,0.025004,0.063182,0.113585,0.000650,0.007845,0.001770,0,0,0,0,0,0,0
3,3,52697150_04.jpg,52697150,52697150_03.jpg,0.000342,0.138134,0.012456,0.204334,0.000092,0.037618,0.001060,0,0,0,0,0,0,0
4,4,52697100_04.jpg,52697100,52697100_03.jpg,0.000437,0.399672,0.045094,0.052942,0.000216,0.444719,0.001802,0,0,0,0,0,0,0
5,5,52697044_04.jpg,52697044,52697044_03.jpg,0.000829,0.011280,0.189606,0.069129,0.002842,0.013522,0.003495,0,0,1,0,0,0,0
6,6,52696984_04.jpg,52696984,52696984_03.jpg,0.007712,0.001604,0.915662,0.040553,0.032953,0.001299,0.000256,0,0,1,0,0,0,0
7,7,52696934_04.jpg,52696934,52696934_03.jpg,0.011304,0.000871,0.828741,0.035456,0.038174,0.001459,0.012761,0,0,1,0,0,0,0
8,8,52696760_04.jpg,52696760,52696760_03.jpg,0.000163,0.041919,0.083482,0.094497,0.000388,0.038317,0.002114,0,0,1,0,0,0,0
9,9,52696225_04.jpg,52696225,52696225_03.jpg,0.000109,0.046792,0.002787,0.243244,0.000063,0.010305,0.000323,0,0,0,0,0,0,0


In [85]:
picturescore = picturescore.merge(pictureprobscore_03[["applied_id", "Heavy_Makeup_prob_score_03"]], on = "applied_id", how = 'left')

In [86]:
picturescore

,filename,applied_id,Eyeglasses,Heavy_Makeup,Male,Wearing_Earrings,Wearing_Necktie,Attractive,Wearing_Hat,uid,...,gender,p1_max_consumed_withdraw_amount_sum,P4M2,Wearing_Necktie_prob_score,prob_score,Wearing_Necktie_flag,risk_level,Heavy_Makeup_prob_score,Heavy_Makeup_flag,Heavy_Makeup_prob_score_03
0,52697741_04.jpg,52697741,0,0,1,0,0,0,0,172651854089961475,...,1.0,1386591.0,0.0,0.000709,NaN,0,NaN,0.002846,0,0.002308
1,52697322_04.jpg,52697322,0,0,1,0,0,0,0,186374770674909206,...,1.0,1102400.0,0.0,0.001598,NaN,0,NaN,0.018194,0,0.005363
2,52697214_04.jpg,52697214,0,0,1,0,0,0,0,167115599362146313,...,1.0,1400000.0,0.0,0.015387,0.017151,0,67.0,0.003544,0,0.025004
3,52697150_04.jpg,52697150,0,0,1,0,0,0,0,186369356856573993,...,1.0,1687160.0,0.0,0.020372,0.036182,0,36.0,0.004059,0,0.138134
4,52697100_04.jpg,52697100,0,0,0,0,0,0,0,186356918626041858,...,2.0,1388398.0,0.0,0.000313,0.022760,0,55.0,0.386678,0,0.399672
5,52697044_04.jpg,52697044,0,0,1,0,0,0,0,182106028430344213,...,1.0,887145.0,0.0,0.011732,0.031512,0,41.0,0.007459,0,0.011280
6,52696984_04.jpg,52696984,0,0,1,0,0,0,0,168334694696632323,...,1.0,1223913.0,0.0,0.032821,0.027427,0,47.0,0.008022,0,0.001604
7,52696934_04.jpg,52696934,0,0,1,0,0,0,0,100435216089497600,...,1.0,1799943.0,0.0,0.014031,0.017398,0,66.0,0.003363,0,0.000871
8,52696760_04.jpg,52696760,0,0,1,0,0,0,0,186373077383004271,...,1.0,602890.0,0.0,0.039018,0.014536,0,74.0,0.018532,0,0.041919
9,52696225_04.jpg,52696225,0,0,1,0,0,0,0,181310305209696272,...,1.0,1777347.0,0.0,0.007592,0.031354,0,41.0,0.004365,0,0.046792


In [44]:
label='P4M2'

In [46]:
picturescore=picturescore.rename(columns={'bad3mon_status_ever':'P4M2'})

In [47]:
picturescore.groupby(['Eyeglasses'])['P4M2'].mean()

Eyeglasses
0    0.053626
1    0.037126
Name: P4M2, dtype: float64

In [51]:
picturescore.groupby(['Eyeglasses'])['GBDT_age'].mean()

Eyeglasses
0    29.204724
1    28.549860
Name: GBDT_age, dtype: float64

In [48]:
print(0.053626/0.037126)

1.4444324731993752


In [41]:
picturescore.groupby(['Eyeglasses','active_month'])['P4M2'].mean() 

Eyeglasses  active_month
0           2019-03-01      0.062953
            2019-04-01      0.062303
            2019-05-01      0.048732
            2019-06-01      0.034298
            2019-07-01      0.038501
            2019-08-01      0.053025
            2019-09-01      0.055669
            2019-10-01      0.058512
            2019-11-01      0.056762
1           2019-03-01      0.044444
            2019-04-01      0.044985
            2019-05-01      0.026933
            2019-06-01      0.024746
            2019-07-01      0.036008
            2019-08-01      0.035968
            2019-09-01      0.038680
            2019-10-01      0.040972
            2019-11-01      0.045397
Name: P4M2, dtype: float64

In [38]:
picturescore.groupby(['Eyeglasses'])['P4M2'].count()

Eyeglasses
0    99280
1    15353
Name: P4M2, dtype: int64

In [49]:
print(picturescore.groupby(['Eyeglasses'])['P4M2'].count()/picturescore.shape[0])

Eyeglasses
0    0.866068
1    0.133932
Name: P4M2, dtype: float64


In [34]:
picturescore.groupby(['Attractive'])['P4M2'].mean()

Attractive
0    0.051421
1    0.000000
Name: P4M2, dtype: float64

In [37]:
picturescore.groupby(['Attractive'])['P4M2'].count()

Attractive
0    114623
1        10
Name: P4M2, dtype: int64

In [35]:
picturescore.groupby(['Wearing_Necktie'])['P4M2'].mean()

Wearing_Necktie
0    0.051420
1    0.042553
Name: P4M2, dtype: float64

In [36]:
picturescore.groupby(['Wearing_Necktie'])['P4M2'].count()

Wearing_Necktie
0    114586
1        47
Name: P4M2, dtype: int64

In [66]:
picturescore.groupby(['Male'])['P4M2'].mean()

Male
0    0.046886
1    0.052655
Name: P4M2, dtype: float64

In [67]:
picturescore.groupby(['Male'])['P4M2'].count()

Male
0    24613
1    90020
Name: P4M2, dtype: int64

In [68]:
picturescore.groupby(['Heavy_Makeup'])['P4M2'].mean()

Heavy_Makeup
0    0.05142
1    0.00000
Name: P4M2, dtype: float64

In [69]:
picturescore.groupby(['Heavy_Makeup'])['P4M2'].count()

Heavy_Makeup
0    114624
1         9
Name: P4M2, dtype: int64

In [78]:
picturescore['Heavy_Makeup_flag'] = picturescore['Heavy_Makeup_prob_score'].map(lambda x: 1 if x > 0.4980192 else 0)

In [79]:
picturescore.groupby(['Heavy_Makeup_flag'])['P4M2'].count()

Heavy_Makeup_flag
0    114289
1       344
Name: P4M2, dtype: int64

In [80]:
picturescore.groupby(['Heavy_Makeup_flag'])['P4M2'].mean()

Heavy_Makeup_flag
0    0.051440
1    0.043605
Name: P4M2, dtype: float64

In [87]:
picturescore['Heavy_Makeup_flag_03'] = picturescore['Heavy_Makeup_prob_score_03'].map(lambda x: 1 if x > 0.4980192 else 0)

In [88]:
picturescore.groupby(['Heavy_Makeup_flag_03'])['P4M2'].count()

Heavy_Makeup_flag_03
0    112663
1      1970
Name: P4M2, dtype: int64

In [89]:
picturescore.groupby(['Heavy_Makeup_flag_03'])['P4M2'].mean()

Heavy_Makeup_flag_03
0    0.051658
1    0.037563
Name: P4M2, dtype: float64

In [90]:
1970/(112663+1970)

0.017185278235761083

In [96]:
112663+1970

114633

In [91]:
0.37/0.51

0.7254901960784313

In [52]:
picturescore['Wearing_Necktie_flag'] = picturescore['Wearing_Necktie_prob_score'].map(lambda x: 1 if x > 0.7454451 else 0)

In [53]:
picturescore.groupby(['Wearing_Necktie_flag'])['P4M2'].count()

Wearing_Necktie_flag
0    114366
1       267
Name: P4M2, dtype: int64

In [95]:
267/(267+114366)

0.0023291722278924918

In [54]:
picturescore.groupby(['Wearing_Necktie'])['P4M2'].mean()

Wearing_Necktie
0    0.051420
1    0.042553
Name: P4M2, dtype: float64

In [92]:
0.042553/0.051420

0.8275573706728899

In [93]:
picturescore.groupby(['Eyeglasses'])['P4M2'].mean()

Eyeglasses
0    0.053626
1    0.037126
Name: P4M2, dtype: float64

In [94]:
0.037126/0.053626

0.6923134300525864

In [99]:
print(picturescore_train.shape)
print(picturescore_oot.shape)

(20178, 21)
(57203, 21)


In [56]:
picturescore_train=picturescore[picturescore['active_month'].map(lambda x: True if(x in  ['2019-03-01','2019-04-01','2020-05-01','2020-06-01']) else False)]

In [57]:
picturescore_oot=picturescore[picturescore['active_month'].map(lambda x: True if(x in  ['2019-07-01','2019-08-01','2019-09-01','2019-10-01']) else False)]

In [58]:
step_wise_result=['risk_level','Eyeglasses']

In [59]:
picturescore_train['risk_level']=picturescore_train['risk_level']/100

/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [60]:
picturescore_oot['risk_level']=picturescore_oot['risk_level']/100

/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
model =sm.Logit(picturescore_train[label], sm.add_constant(pd.DataFrame(picturescore_train[step_wise_result]))).fit(disp=0,method='newton')#返回训练好的模型

/home/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [62]:
model.params

const        -1.694998
risk_level   -3.564940
Eyeglasses   -0.325716
dtype: float64

In [28]:
Gains_report(picturescore_oot[label],model.predict(sm.add_constant(pd.DataFrame(picturescore_oot[step_wise_result]))),n_bins=10)

/home/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


,Rank,# Total,Max Score,Min Score,Mean Score,# Non-Bad,% Total Non-Bad,Cum % Total Non-Bad,# Bads,% Total Bads,% Cum % Total Bads,Interval Bads Rate,Bads vs. Non-Bad K-S
0,1,5721.0,0.150505,0.077323,0.098306,5135.0,9.47%,9.47%,586.0,19.72%,19.72%,10.24%,10.26%
1,2,5721.0,0.077323,0.055421,0.065695,5266.0,9.71%,19.18%,455.0,15.31%,35.04%,7.95%,15.86%
2,3,5721.0,0.055421,0.043717,0.049272,5330.0,9.83%,29.01%,391.0,13.16%,48.2%,6.83%,19.19%
3,4,5721.0,0.043717,0.033230,0.038258,5372.0,9.91%,38.91%,349.0,11.75%,59.95%,6.1%,21.03%
4,5,5721.0,0.033230,0.026082,0.029844,5390.0,9.94%,48.85%,331.0,11.14%,71.09%,5.79%,22.24%
5,6,5721.0,0.026082,0.020440,0.023280,5481.0,10.11%,58.96%,240.0,8.08%,79.17%,4.2%,20.21%
6,7,5721.0,0.020440,0.015446,0.017860,5514.0,10.17%,69.13%,207.0,6.97%,86.13%,3.62%,17.01%
7,8,5721.0,0.015446,0.011255,0.013251,5546.0,10.23%,79.35%,175.0,5.89%,92.02%,3.06%,12.67%
8,9,5721.0,0.011255,0.007632,0.009411,5585.0,10.3%,89.65%,136.0,4.58%,96.6%,2.38%,6.95%
9,10,5714.0,0.007632,0.003737,0.006240,5613.0,10.35%,100.0%,101.0,3.4%,100.0%,1.77%,0


In [29]:
Gains_report(picturescore_oot[label],100-picturescore_oot['risk_level'],n_bins=10)

,Rank,# Total,Max Score,Min Score,Mean Score,# Non-Bad,% Total Non-Bad,Cum % Total Non-Bad,# Bads,% Total Bads,% Cum % Total Bads,Interval Bads Rate,Bads vs. Non-Bad K-S
0,1,5721.0,99.99,99.79,99.856742,5144.0,9.49%,9.49%,577.0,19.42%,19.42%,10.09%,9.94%
1,2,5721.0,99.79,99.69,99.738154,5265.0,9.71%,19.19%,456.0,15.35%,34.77%,7.97%,15.58%
2,3,5721.0,99.69,99.62,99.654289,5329.0,9.83%,29.02%,392.0,13.19%,47.96%,6.85%,18.94%
3,4,5721.0,99.62,99.55,99.580582,5368.0,9.9%,38.92%,353.0,11.88%,59.85%,6.17%,20.93%
4,5,5721.0,99.55,99.47,99.509589,5403.0,9.96%,48.88%,318.0,10.7%,70.55%,5.56%,21.67%
5,6,5721.0,99.47,99.40,99.438091,5475.0,10.1%,58.98%,246.0,8.28%,78.83%,4.3%,19.85%
6,7,5721.0,99.40,99.32,99.362944,5522.0,10.18%,69.16%,199.0,6.7%,85.53%,3.48%,16.37%
7,8,5721.0,99.32,99.23,99.277847,5532.0,10.2%,79.36%,189.0,6.36%,91.89%,3.3%,12.53%
8,9,5721.0,99.23,99.13,99.180138,5575.0,10.28%,89.64%,146.0,4.91%,96.8%,2.55%,7.16%
9,10,5714.0,99.13,99.00,99.065254,5619.0,10.36%,100.0%,95.0,3.2%,100.0%,1.66%,0


In [63]:
(19.72-19.42)/19.42

0.015447991761070912

In [64]:
(35.04-34.77)/34.77

0.007765314926660799

In [65]:
(22.24-21.67)/21.67

0.026303645592985543

In [100]:
22.24-21.67

0.5699999999999967